In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts.chat import MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

e:\Programming-Projects\PBL\DevAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv(find_dotenv(), override=True)

True

In [27]:
# Initialize multiple LLMs for debate
llm_mvp_generator = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"),model="gemini-1.5-flash", temperature=0.7)
llm_minimalist = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash", temperature=0.8)
llm_scalability_advocate = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash", temperature=0.8)
llm_ux_focus = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash", temperature=0.8)
llm_final_decision = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash", temperature=0.6)
llm_design_brainstorm = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash", temperature=0.6)
llm_techstack_decider = ChatGoogleGenerativeAI(google_api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash", temperature=0.7)

In [ ]:
# Define system roles
roles = {
    "Minimalist": "Focus on only the core essential features needed for MVP.",
    "Scalability Advocate": "Think about how features can be expanded in the future and balance feasibility.",
    "UX Focus": "Prioritize user experience, ease of use, and modern design choices."
}

# Generate Initial MVP Feature List
def generate_mvp_features(product_description):
    prompt = f"""
    Given the product description: "{product_description}", generate an initial MVP feature list.
    Focus only on essential features needed for a minimal but functional product.
    """
    response = llm_mvp_generator.predict(prompt)
    print(response)
    return response

# Debate on MVP features
def debate_mvp_features(feature_list):
    critiques = {}
    for role, instruction in roles.items():
        prompt = f"""
        Given the MVP feature list:

        {feature_list}

        Provide your critique as a {role}. {instruction}
        List any features that should be removed, added, or modified.
        """
        critiques[role] = globals()[f"llm_{role.lower().replace(' ', '_')}"].predict(prompt)

    print("********** Critiques from Different Perspectives **********")
    print(critiques)
    return critiques

# Finalize MVP after debate
def finalize_mvp(feature_list, critiques):
    prompt = f"""
    Given the initial MVP feature list:
    {feature_list}
    And the critiques from different perspectives:
    {critiques}
    Refine the MVP feature list by keeping only the most important suggestions.
    Output the final MVP features in a structured JSON format, as a list of string (functionalities).
    """
    final_mvp = llm_final_decision.predict(prompt)
    return final_mvp

In [14]:
# Example Run
product_description = "A simple note-taking app"
initial_mvp = generate_mvp_features(product_description)
critiques = debate_mvp_features(initial_mvp)
final_mvp = finalize_mvp(initial_mvp, critiques)
print("Final MVP Feature Set:", final_mvp)

* **Create a new note:**  Ability to add a new note.
* **Edit an existing note:** Ability to modify an existing note.
* **Delete a note:** Ability to remove a note.
* **Save notes:**  Persistence of notes (local storage is sufficient for MVP).
* **View notes:** Ability to see a list of existing notes and open individual notes for viewing.
* **Basic text formatting:**  Bold, italic, potentially bullet points (keep it minimal).

This list focuses on the core functionality of creating, managing, and viewing notes.  More advanced features (like search, tagging, cloud syncing, etc.) can be added later.
********** Critiques from Different Perspectives **********
{'Minimalist': 'My critique, from a minimalist perspective, focuses on ruthlessly stripping away anything non-essential for a truly *minimal* viable product:\n\n**Features to Remove:**\n\n* **Basic text formatting (Bold, italic, bullet points):**  These are distractions from the core note-taking functionality.  Plain text is sufficie

In [24]:
# Brainstorm Design Guidelines
def brainstorm_design_guidelines(product_description):
    prompt = f"""
    Given the product description: "{product_description}", suggest design guidelines and themes.
    Include aspects like color palette, typography, and branding style.
    Do not include any unnecessary details or features. Just focus on design.
    """
    response = llm_design_brainstorm.predict(prompt)
    return response

In [25]:
design_guidelines = brainstorm_design_guidelines(product_description)

In [26]:
print("Design Guidelines:", design_guidelines)

Design Guidelines: ## Design Guidelines: Simple Note-Taking App

**Goal:**  Clean, minimalist, and functional design that prioritizes readability and ease of use.

**Branding Style:**  Modern and understated. Avoid anything overly playful or complex.  Think clean lines and subtle details.

**Color Palette:**

* **Primary:**  A soft, neutral color like a light grey (#F5F5F5) or a very pale blue (#E6F2FF). This forms the background and provides a calming workspace.
* **Secondary:**  A slightly darker shade of the primary color for subtle contrast in elements like dividers or inactive states.
* **Accent:** A single, muted color for highlighting – perhaps a soft teal (#66B2B2) or a gentle green (#A7D1AB).  Use sparingly.
* **Text:**  Dark grey (#333333) for primary text, slightly lighter grey for secondary text (e.g., timestamps).

**Typography:**

* **Body Text:**  A clean, legible sans-serif font like Open Sans, Roboto, or Lato.  Prioritize readability at various sizes.
* **Headings (opt

In [30]:
def decide_tech_stack(product_description, final_mvp, design_guidelines):
    prompt = f"""
    Given the product description: "{product_description}", the finalized MVP feature list: {final_mvp}, and the design guidelines: {design_guidelines},
    suggest the most suitable tech stack.
    Consider frontend and backend technologies, database choices, deployment options, and scalability factors.
    Don't give options; just provide a single tech stack recommendation.
    """
    response = llm_techstack_decider.predict(prompt)
    return response

In [31]:
tech_stack = decide_tech_stack(product_description, final_mvp, design_guidelines)
print("Suggested Tech Stack:", tech_stack)

Suggested Tech Stack: Frontend: React with TypeScript

Backend: Node.js with Express.js

Database:  Local Storage (for MVP, as specified), with a planned migration path to a cloud-based solution like Supabase or Firebase for future scalability.  

Deployment: Netlify or Vercel (for ease of deployment and automatic scaling).
